In [1]:
import sympy as sy
import control as pc

Part of the big question is whether we are simulating the dynamic response, or the steady-state mathematics iterations. Here below is the steady state reponse mathamtical system design, rather than a dynamic diffferential changes with time. Hence, Python's control toolbox cannot solve a steady-state response of a already stable system. However, attempting to perform a fully stable model means that if we want to perform feedback, there's a factor of recalculating stability from that until differential changes become 0 at different time-steps. So for example, if you were to perform feedback after an initial state, you can get a steady state reponse, but this is not the dynamic response, this is solving for the right answer after a number of iterations, and all the initial steady-state answers are wrong for a feedback system. So this gives a steady state response. If we change an input signal also at another point of time, then there is a new steady state response, but the question is whether you want to account for the time dynamics. Because currently, the system is accounting for the digital computing dynamics rather than the actual transient model, so you can't account for how the system dynamically responds and results in a real-time steady-state. So if an input changes in time, you can actually model the time differential change. If we begin considering delays and inputs into the rest of the system, we need to model time as time and computations as computations - and hence we need a time variable input.

There's also the aspect that optics is fundamentally linear and 3D, so how do we account for differential changes in it? What is its dynamic response? There are only dynamic changes on dynamic inputs. What are time changes in optics? You could argue it is near instantaneous. So there's group delay as the time it takes for light to travel through the waveguide, specially if it has been affected in any way. So there is a need to model transient and steady-state responses, but each of these models should be kept separate.

Now, getting a time domain model of a thermo-optic phase shifter response is non-trivial nor necessarily accurate. So, time is a completely new thing in the optical domain for these type of modelled systems. Yes, there is propagation time effects.

Let's consider an MZI, in one arm there's light going over a certain length which incurrs a certain propagation time. Assuming we're only in steady-state mode, when does change occur? You can differtiate phase shift against length for example, but you would be discretizing space-wise. So imagine Pac-Mans moving in a grid of time, according to their position, their properties change, such as phase and this is also dependent in temperature at a given position, but then there must be a differential in temperature and length to calculate a steady state response. So there's this big aspect of time differentials. Could we work based purely on steady-state time differentials? Actually, that could work. There's also the factor of where to add delays, but if we consider every wavelength arm as a system, then we can simply follow the basic principles of control theory.

Hence, the time discretization must occur at a fraction of the time of propagation of the shortest path in the model. 

So our inputs into the control system are the electromagnetic signals, that are time-domain constrained

#### Beamsplitter
\begin{equation}
\begin{bmatrix} E_{out0}(t) \\ E_{out1}(t) \end{bmatrix} =
\begin{bmatrix} r_{ac}& t_{bc} \\  t_{ad}& r_{bd} \end{bmatrix}
\begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix} = 
\begin{bmatrix} \sqrt{r} &  i\sqrt{1-r} \\  i \sqrt{1-r} & \sqrt{r} \end{bmatrix}
\begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix}  =
\textbf{B}\textbf{E(t)}
\end{equation}

#### Phase Shifter on one arm

Assuming a phase shifter on top arm
\begin{equation}
\textbf{ $\Phi$ } = \begin{bmatrix} e^{i\phi} & 0 \\ 0 & 1\end{bmatrix}
\end{equation}

Note we're not yet taking into account Linear and LinearGroup delay, or the quantum probability variable within the design of the device.

Hence the MZI configuration is:

\begin{equation}
M\textbf{E(t)}  = \textbf{B} \textbf{ $\Phi$ } \textbf{B}\textbf{E(t)} 
\end{equation}

\begin{equation}
M\textbf{E(t)}  = \begin{bmatrix} \sqrt{r} &  i\sqrt{1-r} \\  i \sqrt{1-r} & \sqrt{r} \end{bmatrix} \begin{bmatrix} e^{i\phi} & 0 \\ 0 & 1\end{bmatrix} \begin{bmatrix} \sqrt{r} &  i\sqrt{1-r} \\  i \sqrt{1-r} & \sqrt{r} \end{bmatrix}\begin{bmatrix} E_{in0}(t) \\ E_{in1}(t) \end{bmatrix} = \begin{bmatrix} E_{out0}(t) \\ E_{out1}(t) \end{bmatrix} 
\end{equation}

Now we could just crunch some matrices together and input our time signals, but what if we wanted to leverage classical control system theory with regards to MIMO inputs and outputs, so that it is easier to design this type of systems togther and include feedback and nonlinearity back into the classical control models?

The standard way to start is to create the differential equations for this type of system and we can use the fast linear properties of SymPy to create a single MZI component, and the quantum probability variable could be added into this later. Right now let´s consider that we've only got differential equations, with equal reflectivity on both sides of the matrix:

In [2]:
t = sy.Symbol("t")
E0 = sy.Function("E_0")(t)
E0_s = sy.Symbol("E_0")
dE0_s = sy.Symbol("dE_0")
E1 = sy.Function("E_1")(t)
E1_s = sy.Symbol("E_1")
dE1_s = sy.Symbol("dE_1")
phi = sy.Function("\phi")(t)
r_0, r_1 = sy.symbols("r_0 r_1")
B_0  = sy.Matrix([[sy.sqrt(r_0), sy.I*sy.sqrt(1 - r_0)],
				[sy.I*sy.sqrt(1 - r_0), sy.sqrt(r_0)]])
B_1  = sy.Matrix([[sy.sqrt(r_1), sy.I*sy.sqrt(1 - r_1)],
				[sy.I*sy.sqrt(1 - r_1), sy.sqrt(r_1)]])
Phi = sy.Matrix([[sy.exp(sy.I * phi), 0],
				[0, 1]])
E = sy.Matrix([[E0], [E1]])

out = B_0 * Phi * B_1 * E
out

Matrix([
[(sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - sqrt(1 - r_0)*sqrt(1 - r_1))*E_0(t) + (I*sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + I*sqrt(r_1)*sqrt(1 - r_0))*E_1(t)],
[(sqrt(r_0)*sqrt(r_1) - sqrt(1 - r_0)*sqrt(1 - r_1)*exp(I*\phi(t)))*E_1(t) + (I*sqrt(r_0)*sqrt(1 - r_1) + I*sqrt(r_1)*sqrt(1 - r_0)*exp(I*\phi(t)))*E_0(t)]])

In [3]:
differential_time_response = out.diff(t)
differential_time_response = differential_time_response.replace(E0.diff(t), dE0_s)
differential_time_response = differential_time_response.replace(E1.diff(t), dE1_s)
differential_time_response

Matrix([
[        dE_0*(sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - sqrt(1 - r_0)*sqrt(1 - r_1)) + dE_1*(I*sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + I*sqrt(r_1)*sqrt(1 - r_0)) + I*sqrt(r_0)*sqrt(r_1)*E_0(t)*exp(I*\phi(t))*Derivative(\phi(t), t) - sqrt(r_0)*sqrt(1 - r_1)*E_1(t)*exp(I*\phi(t))*Derivative(\phi(t), t)],
[dE_0*(I*sqrt(r_0)*sqrt(1 - r_1) + I*sqrt(r_1)*sqrt(1 - r_0)*exp(I*\phi(t))) + dE_1*(sqrt(r_0)*sqrt(r_1) - sqrt(1 - r_0)*sqrt(1 - r_1)*exp(I*\phi(t))) - sqrt(r_1)*sqrt(1 - r_0)*E_0(t)*exp(I*\phi(t))*Derivative(\phi(t), t) - I*sqrt(1 - r_0)*sqrt(1 - r_1)*E_1(t)*exp(I*\phi(t))*Derivative(\phi(t), t)]])

The problem becomes an equilibrium condition to solve. One could argue that when the output is in equilibrium or in steady state:

\begin{equation}
\begin{bmatrix} \frac{dE_{out0}}{dt}  \\ \frac{dE_{out1}}{dt} \end{bmatrix}  = 0
\end{equation}

In [4]:
# Note we're only expecting one solution
dE0_dt = sy.solve(differential_time_response[0], dE0_s)[0]
dE0_dt = dE0_dt.replace(E0, E0_s)
dE0_dt = dE0_dt.replace(E1, E1_s)
dE0_dt

(-I*E_0*sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t))*Derivative(\phi(t), t) + E_1*sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t))*Derivative(\phi(t), t) - I*dE_1*sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) - I*dE_1*sqrt(r_1)*sqrt(1 - r_0))/(sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - sqrt(1 - r_0)*sqrt(1 - r_1))

In [5]:
# Note we're only expecting one solution
dE1_dt = sy.solve(differential_time_response[0], dE1_s)[0]
dE1_dt = dE1_dt.replace(E0, E0_s)
dE1_dt = dE1_dt.replace(E1, E1_s)
# dE1_dt = dE1_dt.replace(dE0_s, dE0_dt) # Cannot be solved canonically as it is a complementary system.
dE1_dt

I*(I*E_0*sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t))*Derivative(\phi(t), t) - E_1*sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t))*Derivative(\phi(t), t) + dE_0*sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - dE_0*sqrt(1 - r_0)*sqrt(1 - r_1))/(sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + sqrt(r_1)*sqrt(1 - r_0))

Note that this is a complemenetary function with regards to the other state variable.

Hence, we can write in the nonlinear state-space equations:

\begin{equation}
\dot{\textbf{x}} = \textbf{A} \textbf{x} +  \textbf{B} \textbf{u}
\end{equation}

\begin{equation}
\textbf{y} = \textbf{C} \textbf{x} + \textbf{D} \textbf{u}
\end{equation}

In [6]:
A, B = sy.linear_eq_to_matrix([dE0_dt, dE1_dt], [E0_s, E1_s])
B = sy.simplify(B)
A = sy.simplify(A)

In [7]:
A

Matrix([
[-I*sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t))*Derivative(\phi(t), t)/(sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - sqrt(1 - r_0)*sqrt(1 - r_1)),    sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t))*Derivative(\phi(t), t)/(sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - sqrt(1 - r_0)*sqrt(1 - r_1))],
[  -sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t))*Derivative(\phi(t), t)/(sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + sqrt(r_1)*sqrt(1 - r_0)), -I*sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t))*Derivative(\phi(t), t)/(sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + sqrt(r_1)*sqrt(1 - r_0))]])

In [8]:
B

Matrix([
[ I*dE_1*(sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + sqrt(r_1)*sqrt(1 - r_0))/(sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) - sqrt(1 - r_0)*sqrt(1 - r_1))],
[I*dE_0*(-sqrt(r_0)*sqrt(r_1)*exp(I*\phi(t)) + sqrt(1 - r_0)*sqrt(1 - r_1))/(sqrt(r_0)*sqrt(1 - r_1)*exp(I*\phi(t)) + sqrt(r_1)*sqrt(1 - r_0))]])

Note that the differential state inputs are coupled into the state space system for the opposite original state variables, hence are coupled together and this is the nonlinear represenatation. We can no longer set the differntials inputs to 0, since they are our effective state changes. However, we know have a nonlinear system we have to deal with. It is both nonlinear in $\phi$ and in $\frac{dE_0}{dt} * e^{\phi}$. Note that this system should always be oscillating, that is the nature of light. So let us redefine what a steady state of a system is, because there will be none using standard control theory in optics. But is there a steady state in an MZI? That is a different question. But there is a steady state in an MZI for a given wavelength and is fully dependent on the phase input.

We're also dealing with complementary systems.

### Thermo-optic Phase Shifter Model